In [1]:
import numpy as np
import pandas as pd
import glob

In [2]:
import os
print(os.getcwd())

/home/demo/Desktop/Pi-Pupil-Detection/Pixel Error


In [3]:
cols = ['x', 'y']
# For one participant multiple data recording parts were used in LPW 
#1 - [1,4,9]   2 - [4,10,13]   3 - [16,19,21]  4 - [1,2,12]  5 - [6,10,11]
#6 - [2,5,13]  7 - [15,18,21]  8 - [2,7,9]  9 - [16,17,18]  10 - [1,8,11]

PATICIPANT_PART = 9
PATICIPANT_NUM = 8

canny_blur = [[25, 9], [18, 21], [21, 13]]

path = rf"dataset/LPW_example/LPW_subject{PATICIPANT_NUM}"

print("Participant number: ", PATICIPANT_NUM, "  -  ", " part: ", PATICIPANT_PART)

Participant number:  8   -    part:  9


In [4]:
#detection_rate_result = pd.DataFrame(columns = ['PATICIPANT_NUM', 'PATICIPANT_PART', 'CANNY_VALUE', 'BLUR_VALUE', 'DETECTION_RATE_5PIX', 'DETECTION_RATE_10PIX', 'FRAME_SIZE'])
detection_rate_result = pd.read_csv(path + rf'/detect_participant{PATICIPANT_NUM}.csv', delimiter=',')

In [5]:
for CANNY, BLUR in canny_blur:
    print("CANNY: ", CANNY, "  -  ", " BLUR: ", BLUR)

    detection_observed = pd.read_csv(path + rf'/320Experiment_{PATICIPANT_PART}_canny_{CANNY}_blur_{BLUR}.csv', delimiter='\t')
    
    detection_actual = pd.read_csv(path + rf'/{PATICIPANT_PART}.txt', delim_whitespace= True, header=None)
    detection_actual.columns = cols
    detection_actual[cols] = detection_actual[cols].astype('float')
    detection_observed[cols] = detection_observed[cols].astype('float')

    detection_actual = detection_actual/2
    
    observed_num = len(detection_observed)
    misidentified = len(detection_observed[(detection_observed.extra != '[]') & (detection_observed.x != 0)])
    print(misidentified)
    
    notidentified = len(detection_observed[(detection_observed.x == 0)])
    print(notidentified)
    
    identified = ((observed_num - misidentified - notidentified) * 100) / observed_num
    print(identified)
    
    detection_observed[cols] - detection_actual[cols] 
    
    # Error calculation (MSE)
    
    evaluation = []
    
    for data_o, data_a in zip(detection_observed.values, detection_actual.values):
        #print(np.linalg.norm([data_o[0] , data_a[0]], 1))
        ecu_dist = np.sqrt(np.square(data_o[0] - data_a[0]) + np.square(data_o[1] - data_a[1]))
        evaluation.append(ecu_dist)
        #np.linalg.norm([data_o[0], data_o[1]], [data_a[0], data_a[1]], axis=1)
    
    filter_evaluation =  list(filter(lambda value: value < 100, evaluation))
    
    error_evaluation_5pix =  list(filter(lambda value: value < 5, evaluation))
    tolerated_detection_5pix = (len(error_evaluation_5pix) * 100) / observed_num
    
    error_evaluation_10pix =  list(filter(lambda value: value < 10, evaluation))
    tolerated_detection_10pix = (len(error_evaluation_10pix) * 100) / observed_num
    
    
    df_experiment = pd.DataFrame(filter_evaluation, columns=["Error_Euclidean"])
    
    df_experiment.to_csv(path + f"/320participant{PATICIPANT_NUM}_part_{PATICIPANT_PART}_error__canny_{CANNY}_blur_{BLUR}.csv",
                        sep=',', index=False)
    
    detection_rate_result.loc[len(detection_rate_result.index)] = [PATICIPANT_NUM, PATICIPANT_PART, CANNY, BLUR,  tolerated_detection_5pix, tolerated_detection_10pix, '320x240']
    print(detection_rate_result)
    print("")


CANNY:  25   -    BLUR:  9
0
1995
0.25
    PATICIPANT_NUM  PATICIPANT_PART  CANNY_VALUE  BLUR_VALUE  \
0                8                2           25           9   
1                8                2           18          21   
2                8                2           21          13   
3                8                7           25           9   
4                8                7           18          21   
5                8                7           21          13   
6                8                9           25           9   
7                8                9           18          21   
8                8                9           21          13   
9                8                2           25           9   
10               8                2           18          21   
11               8                2           21          13   
12               8                7           25           9   
13               8                7           18          21   
1

/tmp/ipykernel_5446/2881422461.py:6: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  detection_actual = pd.read_csv(path + rf'/{PATICIPANT_PART}.txt', delim_whitespace= True, header=None)
/tmp/ipykernel_5446/2881422461.py:6: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  detection_actual = pd.read_csv(path + rf'/{PATICIPANT_PART}.txt', delim_whitespace= True, header=None)


    PATICIPANT_NUM  PATICIPANT_PART  CANNY_VALUE  BLUR_VALUE  \
0                8                2           25           9   
1                8                2           18          21   
2                8                2           21          13   
3                8                7           25           9   
4                8                7           18          21   
5                8                7           21          13   
6                8                9           25           9   
7                8                9           18          21   
8                8                9           21          13   
9                8                2           25           9   
10               8                2           18          21   
11               8                2           21          13   
12               8                7           25           9   
13               8                7           18          21   
14               8                7     

/tmp/ipykernel_5446/2881422461.py:6: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  detection_actual = pd.read_csv(path + rf'/{PATICIPANT_PART}.txt', delim_whitespace= True, header=None)


In [6]:
detection_rate_result

,PATICIPANT_NUM,PATICIPANT_PART,CANNY_VALUE,BLUR_VALUE,DETECTION_RATE_5PIX,DETECTION_RATE_10PIX,FRAME_SIZE
0,8,2,25,9,73.75,80.45,480x480
1,8,2,18,21,88.65,93.35,480x480
2,8,2,21,13,85.25,90.70,480x480
3,8,7,25,9,81.55,84.60,480x480
4,8,7,18,21,88.15,91.10,480x480
5,8,7,21,13,87.50,90.15,480x480
6,8,9,25,9,52.10,57.20,480x480
7,8,9,18,21,56.40,63.10,480x480
8,8,9,21,13,58.10,64.15,480x480
9,8,2,25,9,73.25,73.55,320x240


In [7]:
detection_rate_result.to_csv(path + f"/detect_participant{PATICIPANT_NUM}.csv",
                     sep=',', index=False)

In [8]:
print(f"mean - {np.mean(filter_evaluation)}, median - {np.median(filter_evaluation)}, STD - {np.std(filter_evaluation)}")

mean - 16.227629666778764, median - 15.515137718726809, STD - 3.793480282965026


In [9]:
num_not_evaluated = len(evaluation) - len(filter_evaluation)
#max(filter_evaluation)

In [10]:
#for data in filter_evaluation:
#    print(data)
df_experiment = pd.DataFrame(filter_evaluation, columns=["Error_Euclidean"])

df_experiment.to_csv(path + f"/320participant{PATICIPANT_NUM}_part_{PATICIPANT_PART}_error__canny_{CANNY}_blur_{BLUR}.csv",
                     sep=',', index=False)